<a href="https://colab.research.google.com/github/srt3000/Solar-panel-detection/blob/main/Original_solar_YoloV3_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Use the Nvidia GPU if possible

In [ ]:
# Check if NVIDIA GPU is enabled
!nvidia-smi

Sat Dec  4 00:07:13 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.44       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   47C    P8    31W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

****Connect google drive in which the dataset is stored****

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
!ln -s /content/gdrive/My\ Drive/ /mydrive


Mounted at /content/gdrive


**1) Clone the Darknet repository**



In [ ]:
!git clone https://github.com/AlexeyAB/darknet


Cloning into 'darknet'...
remote: Enumerating objects: 15376, done.
remote: Total 15376 (delta 0), reused 0 (delta 0), pack-reused 15376
Receiving objects: 100% (15376/15376), 14.01 MiB | 18.11 MiB/s, done.
Resolving deltas: 100% (10339/10339), done.


**2) Compile Darknet using Nvidia GPU**


For this we have to make some changes to the make file in the darknet


In [ ]:
# change makefile to have GPU and OPENCV enabled
%cd darknet
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile
!make

/content/darknet
mkdir -p ./obj/
mkdir -p backup
chmod +x *.sh
g++ -std=c++11 -std=c++11 -Iinclude/ -I3rdparty/stb/include -DOPENCV `pkg-config --cflags opencv4 2> /dev/null || pkg-config --cflags opencv` -DGPU -I/usr/local/cuda/include/ -DCUDNN -Wall -Wfatal-errors -Wno-unused-result -Wno-unknown-pragmas -fPIC -Ofast -DOPENCV -DGPU -DCUDNN -I/usr/local/cudnn/include -c ./src/image_opencv.cpp -o obj/image_opencv.o
./src/image_opencv.cpp: In function ‘void draw_detections_cv_v3(void**, detection*, int, float, char**, image**, int, int)’:
./src/image_opencv.cpp:946:23: warning: variable ‘rgb’ set but not used [-Wunused-but-set-variable]
                 float rgb[3];
                       ^~~
./src/image_opencv.cpp: In function ‘void draw_train_loss(char*, void**, int, float, float, int, int, float, int, char*, float, int, int, double)’:
./src/image_opencv.cpp:1147:13: warning: this ‘if’ clause does not guard... [-Wmisleading-indentation]
             if (iteration_old == 0)
           

**3) Configure Darknet network for training YOLO V3**

Configuration file is prepared and can be edited using the notepad

In [ ]:
!cp cfg/yolov3.cfg cfg/yolov3_training.cfg

On lines 610, 696 and 783, the no. of classes are changed from 80 to 1 since we are working on only one class - Solar panel.


On lines 603, 689 and 776, the no. of filters are changed from 255 to 18.

In [ ]:
!sed -i 's/batch=1/batch=64/' cfg/yolov3_training.cfg
!sed -i 's/subdivisions=1/subdivisions=16/' cfg/yolov3_training.cfg
!sed -i 's/max_batches = 500200/max_batches = 4000/' cfg/yolov3_training.cfg
!sed -i '610 s@classes=80@classes=1@' cfg/yolov3_training.cfg
!sed -i '696 s@classes=80@classes=1@' cfg/yolov3_training.cfg
!sed -i '783 s@classes=80@classes=1@' cfg/yolov3_training.cfg
!sed -i '603 s@filters=255@filters=18@' cfg/yolov3_training.cfg
!sed -i '689 s@filters=255@filters=18@' cfg/yolov3_training.cfg
!sed -i '776 s@filters=255@filters=18@' cfg/yolov3_training.cfg

Create a folder with name yolov3 

In [ ]:
# Create folder on google drive so that we can save there the weights
!mkdir "/mydrive/yolov3"

mkdir: cannot create directory ‘/mydrive/yolov3’: File exists


Train the model with the class name Solar panel as given during labelling the image


In [ ]:
!echo "Solar panel" > data/obj.names
!echo -e 'classes= 1\ntrain  = data/train.txt\nvalid  = data/test.txt\nnames = data/obj.names\nbackup = /mydrive/yolov3' > data/obj.data
!mkdir data/obj

Download the pretrained darknet 53 model 


In [ ]:
# Download weights darknet model 53
!wget https://pjreddie.com/media/files/darknet53.conv.74

--2021-12-04 00:09:09--  https://pjreddie.com/media/files/darknet53.conv.74
Resolving pjreddie.com (pjreddie.com)... 128.208.4.108
Connecting to pjreddie.com (pjreddie.com)|128.208.4.108|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 162482580 (155M) [application/octet-stream]
Saving to: ‘darknet53.conv.74’

darknet53.conv.74   100%[===================>] 154.96M  56.2MB/s    in 2.8s    

2021-12-04 00:09:12 (56.2 MB/s) - ‘darknet53.conv.74’ saved [162482580/162482580]



**4) Extract Images**

The images need to be inside a zip archive called "images.zip" and they need to be inside the folder "yolov3" on Google Drive

In [ ]:
#unzip the images.zip to data/obj folder
!unzip /mydrive/Datascience/yolov3/images.zip -d data/obj

Archive:  /mydrive/Datascience/yolov3/images.zip
  inflating: data/obj/classes.txt    
  inflating: data/obj/image1.jpg     
  inflating: data/obj/image1.txt     
  inflating: data/obj/image10.jpg    
  inflating: data/obj/image10.txt    
  inflating: data/obj/image1000.jpg  
  inflating: data/obj/image1000.txt  
  inflating: data/obj/image101.jpg   
  inflating: data/obj/image101.txt   
  inflating: data/obj/image102.jpg   
  inflating: data/obj/image102.txt   
  inflating: data/obj/image103.jpg   
  inflating: data/obj/image103.txt   
  inflating: data/obj/image104.jpg   
  inflating: data/obj/image104.txt   
  inflating: data/obj/image105.jpg   
  inflating: data/obj/image105.txt   
  inflating: data/obj/image106.jpg   
  inflating: data/obj/image106.txt   
  inflating: data/obj/image107.jpg   
  inflating: data/obj/image107.txt   
  inflating: data/obj/image108.jpg   
  inflating: data/obj/image108.txt   
  inflating: data/obj/image109.jpg   
  inflating: data/obj/image109.txt   
 

Read the txt files one by one in an array

In [ ]:
# Convert the class index on the .txt files. Since there is only one class, it's supposed to be class 0.

import glob
import os
import re

txt_file_paths = glob.glob(r"data/obj/*.txt")
for i, file_path in enumerate(txt_file_paths):
    # get image size
    with open(file_path, "r") as f_o:
        lines = f_o.readlines()

        text_converted = []
        for line in lines:
            print(line)
            numbers = re.findall("[0-9.]+", line)
            print(numbers)
            if numbers:

                # Define coordinates
                text = "{} {} {} {} {}".format(0, numbers[1], numbers[2], numbers[3], numbers[4])
                text_converted.append(text)
                print(i, file_path)
                print(text)
        # Write file
        with open(file_path, 'w') as fp:
            for item in text_converted:
                fp.writelines("%s\n" % item)

16 0.210987 0.696429 0.028662 0.063187

['16', '0.210987', '0.696429', '0.028662', '0.063187']
0 data/obj/image167.txt
0 0.210987 0.696429 0.028662 0.063187
16 0.494825 0.228709 0.031051 0.028846

['16', '0.494825', '0.228709', '0.031051', '0.028846']
0 data/obj/image167.txt
0 0.494825 0.228709 0.031051 0.028846
16 0.562500 0.511676 0.026274 0.078297

['16', '0.562500', '0.511676', '0.026274', '0.078297']
0 data/obj/image167.txt
0 0.562500 0.511676 0.026274 0.078297
16 0.344347 0.065247 0.021497 0.026099

['16', '0.344347', '0.065247', '0.021497', '0.026099']
0 data/obj/image167.txt
0 0.344347 0.065247 0.021497 0.026099
16 0.455414 0.353022 0.167197 0.112637

['16', '0.455414', '0.353022', '0.167197', '0.112637']
1 data/obj/image244.txt
0 0.455414 0.353022 0.167197 0.112637
16 0.523885 0.553571 0.030255 0.118132

['16', '0.523885', '0.553571', '0.030255', '0.118132']
2 data/obj/image189.txt
0 0.523885 0.553571 0.030255 0.118132
16 0.719745 0.668956 0.063694 0.049451

['16', '0.719745',

Print the image list

In [ ]:
import glob
images_list = glob.glob("data/obj/*.jpg")
print(images_list)

['data/obj/image105.jpg', 'data/obj/image111.jpg', 'data/obj/image189.jpg', 'data/obj/image227.jpg', 'data/obj/image173.jpg', 'data/obj/image244.jpg', 'data/obj/image103.jpg', 'data/obj/image113.jpg', 'data/obj/image138.jpg', 'data/obj/image101.jpg', 'data/obj/image25.jpg', 'data/obj/image137.jpg', 'data/obj/image253.jpg', 'data/obj/image148.jpg', 'data/obj/image24.jpg', 'data/obj/image169.jpg', 'data/obj/image190.jpg', 'data/obj/image171.jpg', 'data/obj/image191.jpg', 'data/obj/image132.jpg', 'data/obj/image118.jpg', 'data/obj/image13.jpg', 'data/obj/image112.jpg', 'data/obj/image183.jpg', 'data/obj/image134.jpg', 'data/obj/image145.jpg', 'data/obj/image166.jpg', 'data/obj/image140.jpg', 'data/obj/image229.jpg', 'data/obj/image114.jpg', 'data/obj/image157.jpg', 'data/obj/image141.jpg', 'data/obj/image233.jpg', 'data/obj/image119.jpg', 'data/obj/image154.jpg', 'data/obj/image133.jpg', 'data/obj/image217.jpg', 'data/obj/image116.jpg', 'data/obj/image182.jpg', 'data/obj/image163.jpg', 'd

In [ ]:
#Create training.txt file
file = open("data/train.txt", "w") 
file.write("\n".join(images_list)) 
file.close() 

**6) Start the training**


Each 100 iterations, the custom object detector is going to be updated and saved in Google drive inside the folder “yolov3”.

 “yolov3_training_last.weights” is the final output weight 

 “yolov3_training__1000.weights”, “yolov3_training_2000.weights” and so on are the backup weights made by the darknet the model for each 1000 iterations.

In [ ]:
# Start the training
!./darknet detector train data/obj.data cfg/yolov3_training.cfg darknet53.conv.74 -dont_show

Streaming output truncated to the last 5000 lines.
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 94 Avg (IOU: 0.463097), count: 3, class_loss = 0.867779, iou_loss = 1.121472, total_loss = 1.989251 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 106 Avg (IOU: 0.000000), count: 1, class_loss = 0.013524, iou_loss = 0.000000, total_loss = 0.013524 
 total_bbox = 50640, rewritten_bbox = 0.051343 % 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 82 Avg (IOU: 0.000000), count: 1, class_loss = 0.006401, iou_loss = 0.000000, total_loss = 0.006401 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 94 Avg (IOU: 0.642042), count: 3, class_loss = 0.838372, iou_loss = 0.420389, total_loss = 1.258761 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 106 Avg (IOU: 0.000000), count: 1, class_loss = 0.012751, iou_loss = 0.000000, total_loss = 0.012751 
 total_bbox = 50643, rewritten_bbox = 0.051340 % 
v3 (mse

**Testing the images**

To test the images


1. load the libraries

In [ ]:
import cv2
import numpy as np
import glob
import random
from google.colab.patches import cv2_imshow
from IPython.display import Image
import IPython
import PIL.Image

 2. Load the saved weight and the configuration file and name the object class to be detected

In [ ]:
# Load Yolo
net = cv2.dnn.readNet(r"/content/gdrive/MyDrive/yolov3/yolov3_training_last.weights", r"/content/gdrive/MyDrive/yolov3/yolov3_testing.cfg")

# Name of the labelled object which is to be detected 
classes = ["Solar panel"]




In [ ]:
!unzip /content/gdrive/MyDrive/yolov3/Test_data.zip -d /content/gdrive/MyDrive/yolov3/Test_data


In [ ]:
ls /content/gdrive/MyDrive/yolov3/Test_data/Solar_Panel/


 3. Load the images  for testing

In [ ]:
# Images path
images_path = glob.glob(r"/content/gdrive/MyDrive/yolov3/Test_data/Solar_Panel/*.jpg")


glob.glob("/content/gdrive/MyDrive/yolov3/Test_data/Solar_Panel/*.jpg")


layer_names = net.getLayerNames()
output_layers = [layer_names[i[0] - 1] for i in net.getUnconnectedOutLayers()]
colors = np.random.uniform(0, 255, size=(len(classes), 3)) 


#random.shuffle(images_path)

5. Detection program 

Confidence and Area of bounding box or the  approx. area of solar panel is printed above the image

In [ ]:
# loop through all the images
 
for img_path in images_path:
    # Loading image
    print(img_path)
    img = cv2.imread(img_path)
    img = cv2.resize(img, None, fx=0.4, fy=0.4)
    height, width, channels = img.shape
    
    # Detecting objects
    blob = cv2.dnn.blobFromImage(img, 0.00392, (416, 416), (0, 0, 0), True, crop=False)

    net.setInput(blob)
    outs = net.forward(output_layers)

    # Showing informations on the screen
    class_ids = []
    confidences = []
    boxes = []
    for out in outs:
        for detection in out:
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]
            if confidence > 0.3:
                # Object detected
                print(class_id)
                center_x = int(detection[0] * width)
                center_y = int(detection[1] * height)
                w = int(detection[2] * width)
                h = int(detection[3] * height)

                # Rectangle coordinates
                x = int(center_x - w / 2)
                y = int(center_y - h / 2)

                boxes.append([x, y, w, h])
                confidences.append(float(confidence))
                class_ids.append(class_id)
                print("Confidence ", confidence)
                print("Area of bounding box, ie. approx. area of solar panel", x*y,"pixels")

    indexes = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)
    print(indexes)
    font = cv2.FONT_HERSHEY_PLAIN
    # Blue color in BGR
    color = (255, 230, 0)
    for i in range(len(boxes)):
        if i in indexes:
            x, y, w, h = boxes[i]
            label = str(classes[class_ids[i]])
            #color = colors[class_ids[i]]
            cv2.rectangle(img, (x, y), (x + w, y + h), color, 1)
            cv2.putText(img, label, (x, y + 30), font, 1, color, 2)
           
    
    IPython.display.display(PIL.Image.fromarray(img))
    

In [ ]:
print(img_path)

In [ ]:
# Images path
images_path = r"D:\Uni\CE888- Data science\Solar\Solar_Panel\*.jpg"





layer_names = net.getLayerNames()
output_layers = [layer_names[i[0] - 1] for i in net.getUnconnectedOutLayers()]
colors = np.random.uniform(0, 255, size=(len(classes), 3)) 


random.shuffle(images_path)

In [ ]:
# loop through all the images
for img_path in images_path:
    # Loading image
    img = cv2.imread(img_path)
    img = cv2.resize(img, None, fx=0.4, fy=0.4)
    height, width, channels = img.shape

    # Detecting objects
    blob = cv2.dnn.blobFromImage(img, 0.00392, (416, 416), (0, 0, 0), True, crop=False)

    net.setInput(blob)
    outs = net.forward(output_layers)

    # Showing informations on the screen
    class_ids = []
    confidences = []
    boxes = []
    for out in outs:
        for detection in out:
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]
            if confidence > 0.3:
                # Object detected
                print(class_id)
                center_x = int(detection[0] * width)
                center_y = int(detection[1] * height)
                w = int(detection[2] * width)
                h = int(detection[3] * height)

                # Rectangle coordinates
                x = int(center_x - w / 2)
                y = int(center_y - h / 2)

                boxes.append([x, y, w, h])
                confidences.append(float(confidence))
                class_ids.append(class_id)
                print("Confidence ", confidence)
                print("Area of bounding box, ie. approx. area of solar panel", x*y,"pixels")

    indexes = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)
    print(indexes)
    font = cv2.FONT_HERSHEY_PLAIN
    # Blue color in BGR
    color = (255, 230, 0)
    for i in range(len(boxes)):
        if i in indexes:
            x, y, w, h = boxes[i]
            label = str(classes[class_ids[i]])
            #color = colors[class_ids[i]]
            cv2.rectangle(img, (x, y), (x + w, y + h), color, 1)
            cv2.putText(img, label, (x, y + 30), font, 1, color, 2)
           
    
    IPython.display.display(PIL.Image.fromarray(img))
    